In [ ]:
using Pkg
Pkg.add("StatsBase")
Pkg.add("Distributions")
#Pkg.add("Ipopt")
#Pkg.add("Juniper")
using Ipopt
using JuMP
using JuMP
using CSV
using DataFrames
using PowerModels
using Ipopt
using Juniper
include("run_optimization.jl")
include("scenario_generation.jl")
include("grid_search.jl")

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.10/Project.toml`
  No Changes to `~/.julia/environments/v1.10/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.10/Project.toml`
  No Changes to `~/.julia/environments/v1.10/Manifest.toml`


sample_cf (generic function with 1 method)

In [23]:
#generate the scenarios
S = 4 # Small right now, but change it later
seasons = rand(["winter", "spring", "summer", "fall"], S)
times = rand(1:24, S)

cf = [sample_cf(s, t) for (s, t) in zip(seasons, times)]
lf = [sample_load(s, t) for (s, t) in zip(seasons, times)]

4-element Vector{Float64}:
 0.7233358673661884
 0.5119107844306429
 0.9027222056596974
 0.9021657756354511

In [ ]:
#run the optimization model
data_path = "DATA/"
baseMVA = 10;

# Run the grid search
step_size = 0.25  # Define the step size for the grid search

# Run the grid search
results = grid_search(step_size, data_path, baseMVA, cf, lf)

This is Ipopt version 3.14.16, running with linear solver MUMPS 5.7.3.

Number of nonzeros in equality constraint Jacobian...:     5468
Number of nonzeros in inequality constraint Jacobian.:      446
Number of nonzeros in Lagrangian Hessian.............:    17784

Total number of variables............................:      778
                     variables with only lower bounds:      706
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:      152
Total number of inequality constraints...............:      302
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:      140
        inequality constraints with only upper bounds:      162

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  1.7999982e-01 2.69e+00 1.71e+00  -1.0 0.00e+00    -  0.00e+00 0.00e+00 

Row,alpha,beta,gamma,objective_cost
,Float64,Float64,Float64,Float64
1,0.0,0.0,1.0,-1.16467e-7
2,0.0,0.25,0.75,1.98921
3,0.0,0.5,0.5,3.97842
4,0.0,0.75,0.25,5.96763
5,0.0,1.0,0.0,7.93745
6,0.25,0.0,0.75,39708.3
7,0.25,0.25,0.5,39709.7
8,0.25,0.5,0.25,39711.2
9,0.25,0.75,0.0,39712.6


02  -1.0 4.74e+02    -  6.94e-01 5.86e-01f  1
 194  6.5115656e+00 7.82e-03 4.76e+02  -1.0 2.64e+03    -  4.02e-01 5.20e-01f  1
 195  6.4902511e+00 3.85e-03 2.17e+02  -1.0 1.00e-01   0.8 5.08e-01 5.07e-01h  1
 196  6.4300839e+00 4.10e-03 3.41e+03  -1.0 1.68e+03    -  6.30e-01 4.92e-01f  1
 197  6.2948244e+00 1.16e-02 1.32e+03  -1.0 3.88e+03    -  4.74e-01 5.32e-01f  1
 198  6.2208085e+00 5.83e-03 2.87e+04  -1.0 3.12e+03    -  7.75e-01 4.97e-01f  1
 199  6.1002074e+00 2.53e-03 1.02e+05  -1.0 8.88e+03    -  1.00e+00 5.67e-01f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 200  5.9438317e+00 3.18e-03 2.19e+05  -1.0 9.27e+04    -  1.00e+00 6.08e-01f  1
 201  5.7728304e+00 3.11e-03 4.65e+05  -1.0 3.64e+04    -  1.00e+00 6.28e-01f  1
 202  5.5947174e+00 3.46e-03 7.78e+05  -1.0 1.35e+04    -  1.00e+00 6.93e-01H  1
 203  5.5177541e+00 2.01e-03 5.28e+05  -1.0 4.16e+03    -  1.00e+00 8.67e-01h  1
 204  5.5091902e+00 5.22e-05 4.12e-04  -1.0 5.52e+02    -  1.00

In [26]:
output_file = "grid_search_results.csv"
CSV.write(output_file, results)

"grid_search_results.csv"